In [ ]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np

To do - image rescalen naar 100x100
confusion matrix
plots
model

In [ ]:
trainDS = tf.data.Dataset.list_files('data/train/*/*', shuffle=False)
test_ds = tf.data.Dataset.list_files('data/test/*/*', shuffle=False)

Check dataset

In [ ]:
for file in trainDS.take(3):
    print(file.numpy())

for file in test_ds.take(3):
    print(file.numpy())

Shuffle function

In [ ]:
# trainDS = trainDS.shuffle(64)
# for file in trainDS.take(5):
#     print(file.numpy())

# testDS = testDS.shuffle(64)
# for file in testDS.take(3):
#     print(file.numpy())

Class Names

In [ ]:
classNames = ["blotch","normal","rot","scab"]

Number of Images in Dataset

In [ ]:
imageCountTrain = len(trainDS)
print (imageCountTrain)
imageCountTest = len(test_ds)
imageCountTest
print (imageCountTest)

In [ ]:
val_size = int(imageCountTrain * 0.2)
train_ds = trainDS.skip(val_size)
val_ds = trainDS.take(val_size)

Images in Train, Test and Val DS

In [ ]:
print (len(train_ds))
print (len(val_ds))
print (len(test_ds))

Getting Images and Labels for each item in the dataset

In [ ]:
def get_label(file_path):
    return tf.strings.split(file_path, os.path.sep)[-2]

PARAMETERS

In [ ]:
img_height = 100
img_width = 100
image_size=(img_height, img_width)
batch_size = 32

In [ ]:
def process_image(file_path):
    label = get_label(file_path)
    
    img = tf.io.read_file(file_path)
    # img = tf.image.resize_with_crop_or_pad(img, 100,100)
    img = tf.image.decode_jpeg(img)
    
    return img, label

In [ ]:
for t in train_ds.take(4):
    print(t.numpy())

In [ ]:
train_ds = train_ds.map(process_image)
for img, label in train_ds.take(3):
    print("Image: ", img)
    print("Label: ",label)

In [ ]:
test_ds = test_ds.map(process_image)
for img, label in test_ds.take(3):
    print("Image: ", img)
    print("Label: ",label)

In [ ]:
val_ds = val_ds.map(process_image)
for img, label in val_ds.take(3):
    print("Image: ", img)
    print("Label: ",label)

Scale function

In [ ]:
def resize_and_rescale(image, label):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, [img_height, img_width])
  image = (image / 255.0)
  return image, label

In [ ]:
train_ds = train_ds.map(resize_and_rescale)
for image, label in train_ds.take(5):
   print ("****Image: ",image.numpy()[0][0])
   print ("****Label: ",label.numpy() )

In [ ]:
test_ds = test_ds.map(resize_and_rescale)
for image, label in test_ds.take(5):
   print ("****Image: ",image.numpy()[0][0])
   print ("****Label: ",label.numpy() )

In [ ]:
val_ds = val_ds.map(resize_and_rescale)
for image, label in val_ds.take(5):
   print ("****Image: ",image.numpy()[0][0])
   print ("****Label: ",label.numpy() )

These operations can be chained but i kept them separate for clarity and possibly oop.

In [ ]:
list_trainDS = tf.data.Dataset.list_files(str('data/train/*/*'))
list_train_ds = list_trainDS.skip(val_size)
list_val_ds = trainDS.take(val_size)
list_test_ds = tf.data.Dataset.list_files(str('data/test/*/*'))

In [ ]:
print (len(list_train_ds))
print (len(list_test_ds))
print (len(list_val_ds))

CAN I CHANGE THIS FROM FILE_PATH LIST TO THE SCALED IMAGES?

In [ ]:
file_path = next(iter(list_train_ds))
image, label = process_image(file_path)

def show(image, label):
  plt.figure()
  plt.imshow(image)
  plt.title(label.numpy().decode('utf-8'))
  plt.axis('off')

show(image, label)

In [ ]:
images_train_ds = list_train_ds.map(process_image)

for image, label in images_train_ds.take(1):
  show(image, label)

Rotate image

In [ ]:
import scipy.ndimage as ndimage

def random_rotate_image(image):
  image = ndimage.rotate(image, np.random.uniform(-30, 30), reshape=False)
  return image

In [ ]:
image, label = next(iter(train_ds))
image = random_rotate_image(image)
show(image, label)

In [ ]:
def tf_random_rotate_image(image, label):
  im_shape = image.shape
  [image,] = tf.py_function(random_rotate_image, [image], [tf.float32])
  image.set_shape(im_shape)
  return image, label

In [ ]:
rot_ds = train_ds.map(tf_random_rotate_image)

for image, label in rot_ds.take(10):
  show(image, label)

resampling kan imbalance in data set wellicht verhelpen: https://www.tensorflow.org/guide/data


Create the model

In [ ]:
num_classes = len(classNames)

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255, input_shape(img_height, img_width, 3)),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

https://www.tensorflow.org/tutorials/load_data/images

In [ ]:
plt.figure(figsize=(10, 10))
for image, label in rot_ds.take(1):
  for i in range(9):
    show(image, label)


# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     # plt.title(class_names[labels[i]])
#     plt.axis("off")

https://www.tensorflow.org/tutorials/keras/overfit_and_underfit

later over and underfit